In [41]:
order_id
activation_time_local: hora local en que se activó el pedido
country_code
store_address
final_status
payment_status
products: número de productos en el pedido
products_total: monto total en el checkout (€)
purchase_total_price: monto que el repartidor pagó en la tienda (€)

SyntaxError: invalid character '€' (U+20AC) (301448624.py, line 8)

In [113]:
from pyspark.sql import SparkSession
from pyspark.sql.functions  import col,when,avg,sum as spark_sum, count,asc,desc
from pyspark.sql.types import IntegerType, StringType, DateType,DoubleType

In [3]:
spark = SparkSession.builder.appName("FakeSpark").getOrCreate()

24/06/28 23:18:15 WARN Utils: Your hostname, LAPTOP-C5IN2DM3 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/06/28 23:18:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/28 23:18:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

In [48]:
df_fake = spark.read.csv("fake_orders_test.csv",header = False)

In [49]:
df_fake.show(5)

+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+
|     _c0|                 _c1|_c2|  _c3|            _c4| _c5|_c6|               _c7|               _c8|
+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+
|33557880|2019-03-10 23:59:...| AR|14200|DeliveredStatus|PAID|  1|              4.54|              8.64|
|33512615|2019-03-10 23:58:...| TR|28725|DeliveredStatus|PAID|  1|              3.76|              3.76|
|33512451|2019-03-10 23:57:...| TR|28725|DeliveredStatus|PAID|  1|2.8600000000000003|2.8600000000000003|
|33530892|2019-03-10 23:57:...| ES|19777| CanceledStatus|PAID|  1|             12.95|                 0|
|33557765|2019-03-10 23:57:...| AR|34565|DeliveredStatus|PAID|  2|2.8600000000000003|6.4799999999999995|
+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+
only showing top 5 rows



In [19]:
nuevos_nombres = ("order_id", "activation_time_local", "nuevo_nombre3","country_code","store_address","final_status","payment_status","products","products_total","purchase_total_price")
df_fake = df_fake.withColumnRenamed("_c0","order_id")

In [191]:
# Supongamos que tienes un DataFrame llamado "df" con una columna "edad" de tipo string que quieres convertir a integer
df_fake = df_fake.withColumn("_c8", col("_c8").cast(DoubleType())) \
                 .withColumn("_c7", col("_c7").cast(DoubleType()))


In [54]:
df_fake.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)



In [53]:
df_fake_clean = df_fake.filter(df_fake["_c4"] != "CanceledStatus")
df_fake_clean.show(5)

+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+
|     _c0|                 _c1|_c2|  _c3|            _c4| _c5|_c6|               _c7|               _c8|
+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+
|33557880|2019-03-10 23:59:...| AR|14200|DeliveredStatus|PAID|  1|              4.54|              8.64|
|33512615|2019-03-10 23:58:...| TR|28725|DeliveredStatus|PAID|  1|              3.76|              3.76|
|33512451|2019-03-10 23:57:...| TR|28725|DeliveredStatus|PAID|  1|2.8600000000000003|2.8600000000000003|
|33557765|2019-03-10 23:57:...| AR|34565|DeliveredStatus|PAID|  2|2.8600000000000003|6.4799999999999995|
|33512273|2019-03-10 23:57:...| TR|63536|DeliveredStatus|PAID|  4|              1.88|              1.96|
+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+
only showing top 5 rows



ejercicio 1

In [62]:
df_fake_clean = df_fake_clean.withColumn("Subautorizados", when ( col("_c8") >= col("_c7"),1).otherwise(0))
df_fake_clean = df_fake_clean.withColumn("Subautorizadosv2", col("_c8") - col("_c7"))
df_fake_clean.show(5)

+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+--------------+-------------------+
|     _c0|                 _c1|_c2|  _c3|            _c4| _c5|_c6|               _c7|               _c8|Subautorizados|   Subautorizadosv2|
+--------+--------------------+---+-----+---------------+----+---+------------------+------------------+--------------+-------------------+
|33557880|2019-03-10 23:59:...| AR|14200|DeliveredStatus|PAID|  1|              4.54|              8.64|             1| 4.1000000000000005|
|33512615|2019-03-10 23:58:...| TR|28725|DeliveredStatus|PAID|  1|              3.76|              3.76|             1|                0.0|
|33512451|2019-03-10 23:57:...| TR|28725|DeliveredStatus|PAID|  1|2.8600000000000003|2.8600000000000003|             1|                0.0|
|33557765|2019-03-10 23:57:...| AR|34565|DeliveredStatus|PAID|  2|2.8600000000000003|6.4799999999999995|             1|  3.619999999999999|
|33512273|2019-03-10

In [68]:
resultado1 = df_fake_clean.agg(((
    spark_sum("Subautorizados")/count("Subautorizados"))* 100).alias("porcentaje")).collect()[0]["porcentaje"]

print(f"El % de pedidos Subautorizados es de: {resultado1:.2f}%")

a = df_fake_clean.filter(col("Subautorizados")==1).count()
b = df_fake_clean.select(col("Subautorizados")).count()
resultado1v2 = (a/b)*100
print(f"El % de pedidos Subautorizados es de: {resultado1v2:.2f}%")

El % de pedidos Subautorizados es de: 86.62%
El % de pedidos Subautorizados es de: 86.62%


In [73]:
a = df_fake_clean.filter(col("Subautorizadosv2")>=0).count()
b = df_fake_clean.select(col("Subautorizadosv2")).count()
resultado1v2 = (a/b)*100
print(f"El % de pedidos Subautorizados es de: {resultado1v2:.2f}%")


El % de pedidos Subautorizados es de: 86.62%


Ejercicio 2

In [84]:
df_fake_clean = df_fake_clean.withColumn("Subautorizados20", when ( col("_c7")*1.2 <= col("_c8"),1).otherwise(0))
df_fake_clean.select(col("_c0"),col("_c7"),col("_c8"),col("Subautorizados"),col("Subautorizados20")).show(5,truncate=False)

+--------+------------------+------------------+--------------+----------------+
|_c0     |_c7               |_c8               |Subautorizados|Subautorizados20|
+--------+------------------+------------------+--------------+----------------+
|33557880|4.54              |8.64              |1             |1               |
|33512615|3.76              |3.76              |1             |0               |
|33512451|2.8600000000000003|2.8600000000000003|1             |0               |
|33557765|2.8600000000000003|6.4799999999999995|1             |1               |
|33512273|1.88              |1.96              |1             |0               |
+--------+------------------+------------------+--------------+----------------+
only showing top 5 rows



In [85]:
a2 = df_fake_clean.filter(col("Subautorizados20")==1).count()
b2 = df_fake_clean.select(col("Subautorizados20")).count()
resultado2 = (a2/b2)*100
print(f"El % de pedidos Subautorizados es de: {resultado2:.2f}%")

El % de pedidos Subautorizados es de: 34.36%


### Ejercicio 3
¿Hay diferencias cuando se dividen por país?

In [119]:
paises = df_fake_clean.groupBy("_c2").agg(((count(when(col("Subautorizados") == 1, True)) / count("Subautorizados")) * 100).alias("v1"),
                                          ((count(when(col("Subautorizados20") == 1, True)) / count("Subautorizados20")) * 100).alias("v20%")).orderBy(asc("v1"))

paises.show(truncate=False)

+---+-----------------+------------------+
|_c2|v1               |v20%              |
+---+-----------------+------------------+
|CI |66.66666666666666|66.66666666666666 |
|PR |68.96551724137932|13.793103448275861|
|CR |73.38551859099805|21.722113502935418|
|FR |76.33860275369709|35.033146353901074|
|IT |77.1122570408568 |33.240777469258234|
|ES |81.05655897566353|30.7809717629102  |
|DO |81.26361655773421|41.17647058823529 |
|PT |82.4074074074074 |41.93121693121693 |
|RO |84.99496475327291|30.31218529707956 |
|GE |88.08510638297872|36.38297872340426 |
|PA |88.34080717488789|35.53811659192825 |
|KE |88.37209302325581|29.069767441860467|
|PE |88.39753818098929|23.72919990882152 |
|UA |89.81204031599019|30.400435848542635|
|TR |89.81465442577516|34.08972804434436 |
|EG |90.94292803970222|38.52357320099256 |
|AR |91.12673549305804|38.90174439302243 |
|GT |91.42300194931774|22.417153996101362|
|EC |91.47045757441137|41.359395824078184|
|MA |92.19088937093277|26.102675343456255|
+---+------

### Ejercicio 4
Para el resto de pedidos que quedarían fuera de la autorización incremental, ¿qué valores serían necesarios para capturar el monto restante?

In [141]:
df_monto = df_fake_clean.withColumn("montofaltante20",(col("_c8") - col("_c7")*1.2))
df_monto = df_monto.filter(col("montofaltante20")>0).select("_c0","montofaltante20")
df_monto.show(truncate = False)

+--------+-------------------+
|_c0     |montofaltante20    |
+--------+-------------------+
|33557880|3.192000000000001  |
|33557765|3.047999999999999  |
|33523444|7.710000000000001  |
|33530379|0.4800000000000004 |
|33509944|0.42600000000000016|
|33503904|4.674000000000002  |
|33521800|1.382              |
|33529187|8.312              |
|33507405|0.5139999999999993 |
|33528930|3.4700000000000006 |
|33506464|6.641999999999998  |
|33520268|0.7120000000000006 |
|33505192|3.926              |
|33520143|0.798              |
|33520135|2.5460000000000003 |
|33519868|11.33              |
|33556356|0.28200000000000003|
|33556296|2.6040000000000005 |
|33503663|2.846              |
|33518904|0.41800000000000015|
+--------+-------------------+
only showing top 20 rows



### Ejercicio 5
¿Qué tiendas son las más problemáticas en términos de pedidos y valor monetario?

In [208]:
df_problema = df_fake_clean.withColumn("faltadinero", when ((col("_c7") - col("_c8")) < 0 ,1).otherwise(0)) \
                                       .withColumn("montofaltante",((col("_c7") - col("_c8"))))
df_problema = df_problema.filter(col("faltadinero") ==1)
df_problema = df_problema.withColumn("montofaltante", col("montofaltante").cast(DoubleType()))
df_problema = df_problema.groupBy("_c3").agg(spark_sum("montofaltante").alias("problemas"))

df_problema.orderBy(asc("problemas")).show(truncate = False)

+-----+-------------------+
|_c3  |problemas          |
+-----+-------------------+
|10264|-1781.1300000000003|
|28669|-1184.449999999999 |
|9855 |-907.03            |
|14455|-791.4100000000001 |
|28671|-769.2999999999995 |
|9854 |-752.7399999999999 |
|17327|-737.3299999999999 |
|10358|-666.04            |
|3063 |-630.0             |
|17326|-602.49            |
|3066 |-596.3999999999999 |
|12513|-548.9             |
|3067 |-545.68            |
|13090|-519.0             |
|17008|-480.6799999999999 |
|1463 |-480.43000000000006|
|51363|-471.5899999999999 |
|17323|-457.90999999999997|
|3062 |-414.70000000000005|
|28712|-408.2500000000001 |
+-----+-------------------+
only showing top 20 rows



In [209]:
spark.stop()

In [210]:
spark

AttributeError: 'NoneType' object has no attribute 'sc'